<a href="https://colab.research.google.com/github/ktamaral/model_vector_comparison/blob/main/bge_embedding_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script generates query embeddings using the bge-m3 model in PyTorch to simulate how Elasticsearch creates them during semantic search. The query embedding emulates the Elasticsearch built-in inference pipeline by adding a prefix to the query, tokenizing with the XLM-Roberta algorithm, applying CLS pooling, and L2-normalizing the vector. The result is a 1024-dimensional embedding that can be compared to a vector generated with the Elasticsearch deployed BGE model.

Generate a token on Hugging Face and add it to the secrets `HF_TOKEN` configuration variable in the notebook settings.



Resources

https://huggingface.co/BAAI/bge-m3

CLS pooling
https://huggingface.co/BAAI/bge-m3/discussions/8#65ca2c33957afc29cedc1310

We use the normalized hidden state of the
special token “[CLS]” for the representation of the
query: eq = norm(Hq[0])
https://arxiv.org/pdf/2402.03216


In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

# BGE-M3 model
model_name = "BAAI/bge-m3"

query = "Radio, TV, and Film Broadcasting History"

# Use Hugging Face tokenizer and model (matches what Elasticsearch uses)
# BAAI/bge-m3 model is built on XLM-RoBERTa architecture
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
# Set to eval mode
model.eval()

# Move model to GPU if available
# https://medium.com/biased-algorithms/mastering-pytorch-to-device-an-advanced-guide-for-efficient-device-management-0290b086f17e
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenization matching Elasticsearch inference config
# https://huggingface.co/transformers/v3.5.1/main_classes/tokenizer.html
inputs = tokenizer(
    query,
    return_tensors="pt",
    max_length=8192,
    # Truncation from the beginning (matches the "first" value)
    truncation=True,
    padding=True
).to(device)


# Run inference and pooling with normalization
# Disable gradient descent
with torch.no_grad():
    outputs = model(**inputs)
    # First token is CLS
    cls_embedding = outputs.last_hidden_state[:, 0]
    normalized = F.normalize(cls_embedding, p=2, dim=1)

# Convert to Python list (to match Elasticsearch expected data structure)
embedding_vector = normalized[0].cpu().tolist()

# Print the vector (1024 floats)
print(embedding_vector)


[-0.012689327821135521, -0.005863994359970093, -0.011758034117519855, 0.05332023650407791, 0.015246723778545856, 0.05450618267059326, -0.02463565394282341, 0.006012905389070511, -0.012675212696194649, -0.03426523879170418, 0.06685935705900192, -0.033846717327833176, -0.02493184059858322, -0.027788322418928146, 0.008008784614503384, -0.014678983017802238, 0.0450933612883091, -0.008503902703523636, -0.010289196856319904, -0.03152567520737648, 0.04021887481212616, -0.03334552422165871, -0.020069722086191177, 0.008610690012574196, 0.04514830559492111, 0.014425993897020817, -0.004150348249822855, -0.04147106409072876, 0.005020053591579199, -0.03602321818470955, -0.019733851775527, 0.008493633009493351, 0.00482903141528368, -0.019483640789985657, -0.05794756859540939, -0.005783999338746071, -0.05397295579314232, 0.0049957893788814545, -0.06491168588399887, -0.080360047519207, -0.04070839285850525, -0.031482331454753876, -0.023654766380786896, -0.04657631367444992, -0.012291178107261658, -0.0

The pytorch model configuration in the script above emulateas the model configuration in Elasticsearch shown after running a GET request on the `_ml/trained_models` API endpoint.

In [ ]:
# GET _ml/trained_models/bge-m3

{
  "count": 1,
  "trained_model_configs": [
    {
      "model_id": "bge-m3",
      "model_type": "pytorch",
      "created_by": "api_user",
      "version": "12.0.0",
      "create_time": 1741371478089,
      "model_size_bytes": 0,
      "estimated_operations": 0,
      "license_level": "platinum",
      "description": "Model BAAI/bge-m3 for task type 'text_embedding'",
      "tags": [],
      "metadata": {
        "per_allocation_memory_bytes": 12377538568,
        "per_deployment_memory_bytes": 2266951712
      },
      "input": {
        "field_names": [
          "text_field"
        ]
      },
      "inference_config": {
        "text_embedding": {
          "vocabulary": {
            "index": ".ml-inference-native-000002"
          },
          "tokenization": {
            "xlm_roberta": {
              "do_lower_case": false,
              "with_special_tokens": true,
              "max_sequence_length": 8192,
              "truncate": "first",
              "span": -1
            }
          },
          "embedding_size": 1024
        }
      },
      "location": {
        "index": {
          "name": ".ml-inference-native-000002"
        }
      }
    }
  ]
}

# Compare embedding with one retrieved from ES

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

# Get a vector from ES
es_vector = [
        -0.012689272873103619,
        -0.005864003207534552,
        -0.011758002452552319,
        0.05332026258111,
        0.015246693976223469,
        0.05450617894530296,
        -0.024635683745145798,
        0.006012951955199242,
        -0.012675218284130096,
        -0.034265194088220596,
        0.06685937941074371,
        -0.03384682908654213,
        -0.024931786581873894,
        -0.02778833545744419,
        0.008008766919374466,
        -0.014679040759801865,
        0.04509337991476059,
        -0.00850401259958744,
        -0.010289154946804047,
        -0.03152568265795708,
        0.04021887853741646,
        -0.033345457166433334,
        -0.02006971649825573,
        0.008610679768025875,
        0.04514823853969574,
        0.014425946399569511,
        -0.004150341730564833,
        -0.04147108644247055,
        0.00502003962174058,
        -0.03602330759167671,
        -0.019733842462301254,
        0.008493599481880665,
        0.004829092882573605,
        -0.019483651965856552,
        -0.057947587221860886,
        -0.0057840729132294655,
        -0.0539729967713356,
        0.004995827563107014,
        -0.06491167843341827,
        -0.08036013692617416,
        -0.04070836678147316,
        -0.03148224577307701,
        -0.023654738441109657,
        -0.046576231718063354,
        -0.012291125021874905,
        -0.02463376149535179,
        -0.009927845560014248,
        -0.0023112983908504248,
        0.009388484992086887,
        -0.011574175208806992,
        -0.02007428929209709,
        0.0033145963679999113,
        0.03059050627052784,
        -0.004072530660778284,
        0.021017462015151978,
        0.053467996418476105,
        -0.034217800945043564,
        0.018370451405644417,
        -0.03896474093198776,
        -0.009170310571789742,
        0.04321084916591644,
        0.02894352190196514,
        -0.0047814855352044106,
        -0.0015484587056562304,
        0.026918631047010422,
        0.05415987968444824,
        0.014450781978666782,
        0.00772583344951272,
        -0.009544986300170422,
        -0.010896817781031132,
        -0.05456038564443588,
        0.0104588083922863,
        0.021206753328442574,
        -0.018202273175120354,
        -0.01951625384390354,
        0.026766173541545868,
        -0.008535041473805904,
        -0.01661689579486847,
        0.05672118067741394,
        -0.007660690229386091,
        -0.030847955495119095,
        -0.03114059939980507,
        0.07620681822299957,
        -0.004578206222504377,
        -0.003542887279763818,
        -0.007331300526857376,
        -0.022525550797581673,
        0.010680614039301872,
        0.005736766383051872,
        -0.014775974676012993,
        0.015652136877179146,
        -0.018693316727876663,
        -0.036384861916303635,
        -0.01469383668154478,
        -0.009717910550534725,
        -0.035570044070482254,
        -0.03433611989021301,
        0.0006714159389957786,
        0.014583735726773739,
        -0.014442703686654568,
        -0.0030417602974921465,
        -0.0051402924582362175,
        0.008429662324488163,
        -0.0076299309730529785,
        0.02206619456410408,
        -0.0269947350025177,
        0.031022729352116585,
        0.015090398490428925,
        -0.03271408751606941,
        -0.005557858385145664,
        0.009014728479087353,
        0.01759573630988598,
        -0.038608063012361526,
        0.03534397482872009,
        -0.01541206892579794,
        -0.0007838672026991844,
        -0.0002177366695832461,
        0.00030088244238868356,
        0.005150306969881058,
        0.013271368108689785,
        0.02191047929227352,
        0.02710275910794735,
        0.05708073824644089,
        -0.02208363078534603,
        0.030775688588619232,
        -0.030483053997159004,
        0.04391244053840637,
        0.033808477222919464,
        0.044997699558734894,
        -0.030176471918821335,
        0.03400002792477608,
        -0.00012166258238721639,
        0.010104081593453884,
        -0.0006476884009316564,
        -0.05360136181116104,
        -0.00765361450612545,
        -0.02388734370470047,
        0.012675030156970024,
        0.03785320743918419,
        -0.04179394617676735,
        0.06189815700054169,
        -0.035134077072143555,
        -0.00290310219861567,
        -0.01204987708479166,
        -0.028945915400981903,
        -0.03786121681332588,
        -0.02686825580894947,
        0.032023120671510696,
        0.027322541922330856,
        -0.017237147316336632,
        -0.017520228400826454,
        -0.00750915240496397,
        0.008566445671021938,
        -0.0334450863301754,
        -0.01667127199470997,
        0.05067348852753639,
        -0.014158637262880802,
        0.016893288120627403,
        -0.03465539962053299,
        0.006561006419360638,
        0.021547844633460045,
        0.02567889913916588,
        -0.048328276723623276,
        -0.019356384873390198,
        0.015320475213229656,
        0.00005317764953360893,
        0.0423256978392601,
        0.024819547310471535,
        -0.01765727438032627,
        0.04565019533038139,
        0.007737462408840656,
        0.002512851497158408,
        0.03144000098109245,
        0.014931046403944492,
        -0.02363142929971218,
        0.018022844567894936,
        0.028732143342494965,
        0.020314211025834084,
        -0.012840203940868378,
        -0.07644270360469818,
        -0.062286630272865295,
        -0.026144661009311676,
        -0.03858731687068939,
        -0.01090348418802023,
        0.00810956209897995,
        -0.0017571734497323632,
        -0.04947815090417862,
        -0.04487480968236923,
        0.012686526402831078,
        0.009332840330898762,
        0.004815806169062853,
        0.007177179679274559,
        -0.0443539172410965,
        -0.003953511361032724,
        -0.00793217122554779,
        -0.05586865171790123,
        0.005749376956373453,
        0.009260757826268673,
        -0.03649143502116203,
        -0.001848263549618423,
        -0.05076988786458969,
        -0.023240964859724045,
        -0.04385964199900627,
        0.05356653779745102,
        -0.023887336254119873,
        -0.019827913492918015,
        -0.0218235794454813,
        -0.03465000167489052,
        -0.005606105085462332,
        -0.012735008262097836,
        -0.012344775721430779,
        -0.05334661528468132,
        0.044296715408563614,
        -0.0769319012761116,
        0.015215646475553513,
        0.03037731721997261,
        0.03373659402132034,
        -0.02436215616762638,
        0.01864691823720932,
        0.014629554003477097,
        0.00745354313403368,
        0.05724108964204788,
        0.02479829266667366,
        -0.03701307997107506,
        0.06611831486225128,
        -0.06794653832912445,
        0.06407549977302551,
        -0.013443723320960999,
        -0.0008915516664274037,
        -0.05516466870903969,
        -0.010171869769692421,
        -0.020726552233099937,
        -0.02572179213166237,
        -0.002717413008213043,
        0.009454912506043911,
        0.012388058006763458,
        -0.0003903450851794332,
        0.008092186413705349,
        0.008163194172084332,
        0.02157462388277054,
        0.004672250710427761,
        -0.01692088134586811,
        -0.044951241463422775,
        0.007227667141705751,
        0.0018482656450942159,
        0.0254187174141407,
        0.01697295717895031,
        -0.006202074699103832,
        -0.04376508295536041,
        -0.019422320649027824,
        0.005584030412137508,
        -0.01020217314362526,
        -0.02017582580447197,
        0.024415908381342888,
        0.07462527602910995,
        0.008869367651641369,
        0.027936454862356186,
        -0.000867039489094168,
        0.006018393207341433,
        -0.024678435176610947,
        0.018408793956041336,
        -0.018010428175330162,
        0.012456352822482586,
        0.01861579902470112,
        0.003155819606035948,
        -0.02637501060962677,
        -0.013999727554619312,
        -0.01902233064174652,
        0.031623926013708115,
        -0.026721764355897903,
        -0.015359011478722095,
        -0.044543493539094925,
        0.05713554099202156,
        0.004035092890262604,
        -0.020424267277121544,
        0.013852648437023163,
        0.07542182505130768,
        -0.03470490872859955,
        -0.012851577252149582,
        -0.0076753003522753716,
        -0.04497573897242546,
        -0.00011808293493231758,
        0.03589707612991333,
        0.017195630818605423,
        -0.006279487628489733,
        -0.07687337696552277,
        -0.008424817584455013,
        0.0033132394310086966,
        -0.026701059192419052,
        -0.06758205592632294,
        0.019724419340491295,
        0.025129932910203934,
        0.03374259173870087,
        -0.018640708178281784,
        -0.028014184907078743,
        0.031772371381521225,
        -0.04552209749817848,
        -0.15231959521770477,
        0.004666747059673071,
        0.004130419343709946,
        0.03885941579937935,
        -0.03085990622639656,
        -0.030963221564888954,
        0.02411147765815258,
        -0.01435142382979393,
        -0.035983145236968994,
        0.033070389181375504,
        -0.0003882727469317615,
        -0.02471745014190674,
        -0.028284769505262375,
        -0.04176720604300499,
        -0.0031668778974562883,
        -0.04035007581114769,
        -0.006162140518426895,
        -0.026364469900727272,
        0.042306143790483475,
        -0.009952339343726635,
        0.024444399401545525,
        -0.03307443484663963,
        -0.0034391460940241814,
        0.050747837871313095,
        0.022936735302209854,
        -0.011388587765395641,
        -0.00041484658140689135,
        -0.03862917050719261,
        -0.03375261276960373,
        0.027821622788906097,
        0.0757879838347435,
        -0.004025560803711414,
        -0.007328374311327934,
        0.00633417721837759,
        0.04653231054544449,
        -0.03727483004331589,
        0.023668548092246056,
        -0.028335796669125557,
        0.01137866172939539,
        -0.009428043849766254,
        -0.03403487801551819,
        0.05027691274881363,
        0.002913319505751133,
        0.029599953442811966,
        0.002098846947774291,
        -0.07166047394275665,
        0.028096003457903862,
        0.0012350110337138176,
        0.024135934188961983,
        -0.006602597888559103,
        -0.023915447294712067,
        -0.009711341932415962,
        -0.0052558802999556065,
        -0.029662540182471275,
        -0.007181704975664616,
        -0.0033307101111859083,
        0.02018735185265541,
        0.09301385283470154,
        -0.011282471939921379,
        0.01971881464123726,
        0.01229830738157034,
        0.025464704260230064,
        0.030523104593157768,
        0.002585667185485363,
        0.0025221197865903378,
        -0.0036377315409481525,
        -0.04417915642261505,
        -0.023649299517273903,
        0.030337607488036156,
        -0.03890034183859825,
        0.009580696932971478,
        0.02936255931854248,
        0.019692324101924896,
        0.03376106172800064,
        -0.02866782806813717,
        0.006783962715417147,
        -0.006333703175187111,
        -0.009934104979038239,
        -0.04680166020989418,
        -0.005302011035382748,
        -0.0281235221773386,
        0.004400803241878748,
        -0.008782885037362576,
        0.0037451498210430145,
        -0.04984147101640701,
        -0.024452706798911095,
        0.015155303291976452,
        0.018742060288786888,
        0.06762689352035522,
        0.19997161626815796,
        -0.010208738967776299,
        0.028684455901384354,
        -0.013063514605164528,
        0.046092357486486435,
        -0.020730357617139816,
        0.022438988089561462,
        0.0004514915344770998,
        -0.02614886313676834,
        -0.028991572558879852,
        0.06093978509306908,
        0.048345230519771576,
        -0.01135243196040392,
        0.0037476506549865007,
        -0.025190526619553566,
        0.012102511711418629,
        -0.03015034832060337,
        0.009548108093440533,
        0.05767704173922539,
        0.015937158837914467,
        0.0057348101399838924,
        0.03951925411820412,
        0.03273247927427292,
        -0.015421472489833832,
        0.04746698588132858,
        -0.006598868407309055,
        -0.004452785011380911,
        0.03416360914707184,
        0.044208116829395294,
        0.0065207104198634624,
        -0.017927512526512146,
        -0.017198869958519936,
        0.005388727877289057,
        0.020562944933772087,
        0.005659531801939011,
        0.05318574234843254,
        0.0006588721880689263,
        -0.04125917702913284,
        0.045774947851896286,
        0.016858218237757683,
        -0.03583075478672981,
        -0.042260244488716125,
        0.0007070336141623557,
        -0.015637917444109917,
        -0.025351794436573982,
        -0.07235690206289291,
        0.006357123143970966,
        -0.0067153251729905605,
        -0.05222121626138687,
        -0.029391251504421234,
        0.012176825664937496,
        0.09297781437635422,
        0.05396561324596405,
        -0.018611611798405647,
        -0.014150508679449558,
        -0.023003291338682175,
        0.006622381042689085,
        0.05275517702102661,
        -0.017947230488061905,
        0.058484867215156555,
        0.05694495141506195,
        -0.009026171639561653,
        0.03259750083088875,
        0.045310601592063904,
        0.01845729537308216,
        0.00017339953046757728,
        0.0024896394461393356,
        -0.029821528121829033,
        -0.007588147651404142,
        0.028649499639868736,
        0.015064490027725697,
        -0.03205035626888275,
        0.001729103154502809,
        -0.011289061047136784,
        0.019108301028609276,
        0.02322271838784218,
        0.0008987270412035286,
        0.061304446309804916,
        -0.04695678502321243,
        0.01389335561543703,
        0.025654787197709084,
        -0.020343050360679626,
        -0.06136319041252136,
        -0.01975863426923752,
        0.006776171270757914,
        -0.010362556204199791,
        0.029272958636283875,
        0.014198743738234043,
        0.0009762772242538631,
        -0.00879306998103857,
        -0.007640823256224394,
        0.01414460875093937,
        -0.026279807090759277,
        -0.0022675914224237204,
        -0.0023000205401331186,
        -0.019882239401340485,
        0.05013797804713249,
        0.015091524459421635,
        -0.005683085415512323,
        0.029994944110512733,
        0.016964955255389214,
        0.04588211700320244,
        -0.0016816894058138132,
        -0.01688346266746521,
        0.04908260703086853,
        0.027443841099739075,
        0.02090393379330635,
        0.018597913905978203,
        -0.026525650173425674,
        -0.0033826311118900776,
        0.040012795478105545,
        0.020460525527596474,
        0.020319536328315735,
        -0.002016705460846424,
        0.03856430947780609,
        0.04106893762946129,
        -0.03287087753415108,
        0.005457080900669098,
        0.0013086146209388971,
        0.017377173528075218,
        0.03754417225718498,
        -0.04746582359075546,
        0.05448712036013603,
        -0.0022936654277145863,
        -0.03886235132813454,
        -0.014280196279287338,
        -0.03172817453742027,
        -0.0280684195458889,
        0.034984465688467026,
        -0.009281132370233536,
        0.036144424229860306,
        0.0029741935431957245,
        0.002357273595407605,
        0.009393029846251011,
        0.04574860259890556,
        0.03884591907262802,
        -0.027680184692144394,
        -0.008592601865530014,
        0.04102661460638046,
        0.011558080092072487,
        -0.007932174950838089,
        0.06661578267812729,
        -0.04232182726264,
        -0.012757000513374805,
        0.0015664229867979884,
        -0.010926280170679092,
        0.012329510413110256,
        0.01583538018167019,
        -0.024814821779727936,
        0.04254165291786194,
        0.03350280225276947,
        0.009777284227311611,
        -0.001226018532179296,
        0.008396707475185394,
        0.034708477556705475,
        0.039038676768541336,
        -0.048698268830776215,
        -0.021528849378228188,
        0.019687393680214882,
        -0.026312923058867455,
        0.015883201733231544,
        0.023597564548254013,
        -0.030560024082660675,
        0.07485300302505493,
        -0.0008656562422402203,
        -0.06306219100952148,
        -0.01579192653298378,
        0.024778109043836594,
        -0.01669285073876381,
        0.01529464777559042,
        -0.03516126051545143,
        -0.04034116864204407,
        -0.060827434062957764,
        0.03200765699148178,
        -0.00625518849119544,
        0.006819344125688076,
        -0.03847073018550873,
        -0.04142434522509575,
        -0.0370279923081398,
        0.03418003022670746,
        0.032395072281360626,
        0.025091901421546936,
        -0.0013311312068253756,
        -0.023160282522439957,
        -0.013246343471109867,
        0.02459612861275673,
        -0.027095723897218704,
        -0.033058129251003265,
        0.019597353413701057,
        -0.0120419105514884,
        0.014899177476763725,
        0.0992414653301239,
        0.004894619341939688,
        -0.026081278920173645,
        0.03321344777941704,
        0.022121209651231766,
        -0.01316055841743946,
        0.015987876802682877,
        -0.003788297064602375,
        -0.0114352498203516,
        0.031068352982401848,
        0.043154679238796234,
        -0.013715132139623165,
        0.034123584628105164,
        -0.03773660212755203,
        0.003837142139673233,
        -0.03240720555186272,
        -0.01334446668624878,
        -0.013361743651330471,
        -0.01214074157178402,
        0.005864008795469999,
        0.03316940367221832,
        -0.007654417771846056,
        0.06783048808574677,
        -0.03476273640990257,
        -0.004237275570631027,
        0.0034361628349870443,
        0.03758203238248825,
        0.02879328839480877,
        -0.037458449602127075,
        0.0016013503773137927,
        -0.03305399417877197,
        -0.01966550573706627,
        0.025716081261634827,
        0.006734977941960096,
        -0.03438742086291313,
        0.03357849642634392,
        0.016272157430648804,
        -0.0067581250332295895,
        0.010539647191762924,
        0.012320082634687424,
        -0.03751048818230629,
        0.03246905654668808,
        -0.008600464090704918,
        0.058675818145275116,
        -0.0033724107779562473,
        -0.016778700053691864,
        -0.004504764452576637,
        -0.01853020489215851,
        0.009008706547319889,
        0.04752518981695175,
        -0.009535614401102066,
        0.04857027903199196,
        -0.01674746721982956,
        0.01933206059038639,
        -0.017824554815888405,
        0.008570762351155281,
        -0.05692865327000618,
        0.04807579889893532,
        0.032012589275836945,
        -0.028472496196627617,
        0.04021785035729408,
        -0.016671616584062576,
        -0.057145096361637115,
        0.00782662071287632,
        -0.013772515580058098,
        -0.005526512861251831,
        -0.016494112089276314,
        -0.054052285850048065,
        0.006900758948177099,
        -0.001503896084614098,
        -0.017825396731495857,
        -0.01501390803605318,
        0.026366790756583214,
        0.000010646758710208815,
        -0.014483854174613953,
        0.02740384452044964,
        0.01891498453915119,
        -0.037156183272600174,
        -0.0017522183479741216,
        0.0330599844455719,
        0.012336168438196182,
        -0.03280503675341606,
        -0.028348825871944427,
        0.03011583350598812,
        0.003667964367195964,
        -0.04353807494044304,
        0.01446920819580555,
        0.003571721026673913,
        0.036122120916843414,
        0.009041673503816128,
        0.060340456664562225,
        -0.004697520285844803,
        0.05741320922970772,
        0.006309452932327986,
        -0.028728339821100235,
        0.03276538476347923,
        0.02309911884367466,
        -0.01420674193650484,
        -0.05120573565363884,
        -0.007695714011788368,
        -0.05366600677371025,
        0.010084244422614574,
        -0.014050052501261234,
        0.006272909231483936,
        0.0007833910058252513,
        -0.016887081786990166,
        0.03358377888798714,
        -0.014064710587263107,
        -0.04052183777093887,
        -0.01650231145322323,
        0.030277598649263382,
        -0.0026164029259234667,
        -0.016389524564146996,
        0.03323070704936981,
        -0.0002774749300442636,
        0.02613857015967369,
        -0.007838403806090355,
        0.05243353545665741,
        0.02269425056874752,
        0.029704490676522255,
        0.013300896622240543,
        0.02074565179646015,
        0.009551308117806911,
        -0.08031607419252396,
        -0.010021446272730827,
        -0.0019591497257351875,
        -0.04416244477033615,
        -0.007593936286866665,
        -0.039591994136571884,
        0.018585296347737312,
        -0.03209203481674194,
        0.017358077690005302,
        -0.01473004836589098,
        -0.0006604264490306377,
        0.015336640179157257,
        0.022161610424518585,
        0.00652296980842948,
        -0.023232314735651016,
        -0.046641454100608826,
        -0.031208382919430733,
        0.013948293402791023,
        0.0521949864923954,
        0.014473825693130493,
        0.017210770398378372,
        0.01819901168346405,
        0.02595578320324421,
        -0.002778872149065137,
        0.011663168668746948,
        -0.035879697650671005,
        -0.004151614848524332,
        -0.0005957101238891482,
        0.037288255989551544,
        -0.0012238690396770835,
        0.0318002924323082,
        -0.05942293629050255,
        -0.021035218611359596,
        0.014314154163002968,
        0.02682487852871418,
        0.021737920120358467,
        -0.03377128392457962,
        0.03720643371343613,
        -0.032277610152959824,
        -0.03411408141255379,
        0.01177938561886549,
        -0.04640890657901764,
        -0.03638995438814163,
        -0.0217985138297081,
        -0.0031077663879841566,
        0.04295561835169792,
        -0.0021019643172621727,
        -0.01807665452361107,
        0.0056962124072015285,
        -0.013946808874607086,
        -0.011494169011712074,
        0.005481558386236429,
        -0.02090483345091343,
        0.011683526448905468,
        0.015032275579869747,
        -0.015182875096797943,
        -0.04593541845679283,
        0.02118626981973648,
        -0.05736662819981575,
        0.028028562664985657,
        -0.052956610918045044,
        0.02315000258386135,
        -0.03030969575047493,
        -0.029863514006137848,
        0.026284147053956985,
        0.006170940585434437,
        -0.0059694647789001465,
        -0.02967611886560917,
        -0.0034815515391528606,
        -0.012517393566668034,
        0.006770925130695105,
        -0.059609655290842056,
        -0.027502000331878662,
        -0.04074202850461006,
        -0.03951063007116318,
        0.044125813990831375,
        -0.05282685533165932,
        0.034017935395240784,
        0.036354999989271164,
        -0.0048441896215081215,
        -0.12272658944129944,
        0.0802215039730072,
        -0.005580997560173273,
        0.007667632307857275,
        -0.013098552823066711,
        0.00539231114089489,
        0.0026191119104623795,
        0.051423683762550354,
        -0.05905947461724281,
        0.007569353561848402,
        0.013423372060060501,
        -0.0021285300608724356,
        -0.0013636783696711063,
        -0.006293564569205046,
        -0.000814428087323904,
        0.011204260401427746,
        0.0020350818522274494,
        0.029575424268841743,
        -0.019264155998826027,
        0.005854926537722349,
        -0.011143320240080357,
        -0.03208782523870468,
        0.06221701577305794,
        -0.0019519126508384943,
        0.05194613337516785,
        -0.01843464933335781,
        0.024588266387581825,
        0.025620045140385628,
        -0.02522876486182213,
        -0.04261725768446922,
        -0.015512038953602314,
        0.00926784798502922,
        0.00608324958011508,
        0.03298884257674217,
        0.017305035144090652,
        0.05473272502422333,
        -0.0004730702203232795,
        -0.020470822229981422,
        0.0051481896080076694,
        0.0011481624096632004,
        0.003676146036013961,
        0.03473895043134689,
        0.011135142296552658,
        -0.047842059284448624,
        -0.039377689361572266,
        -0.017684994265437126,
        -0.03661450743675232,
        0.0037670007441192865,
        0.007914614863693714,
        -0.02712997794151306,
        -0.005316291935741901,
        -0.032759636640548706,
        0.017587805166840553,
        -0.007467862218618393,
        -0.026576630771160126,
        0.012987794354557991,
        -0.03941183537244797,
        0.016734063625335693,
        0.05636250600218773,
        0.01935281977057457,
        -0.02886027842760086,
        -0.019774312153458595,
        0.013510983437299728,
        -0.011749335564672947,
        -0.06754586845636368,
        0.001354763051494956,
        -0.028497347608208656,
        0.009046096354722977,
        -0.030714470893144608,
        0.043898530304431915,
        -0.03346855193376541,
        0.06253065168857574,
        0.009997946210205555,
        -0.04125367850065231,
        -0.01003351155668497,
        0.004034131299704313,
        -0.00181044009514153,
        -0.01272760983556509,
        -0.05231601744890213,
        -0.021974902600049973,
        -0.014042279683053493,
        0.005738580599427223,
        0.016869336366653442,
        -0.0003756995138246566,
        -0.009931322187185287,
        0.023022135719656944,
        -0.030187256634235382,
        -0.0019552218727767467,
        -0.04538758099079132,
        0.0009456249536015093,
        -0.022989915683865547,
        -0.023948054760694504,
        0.016599735245108604,
        0.08977410942316055,
        -0.07212518900632858,
        -0.00008208045619539917,
        -0.004628282506018877,
        0.009594054892659187,
        -0.02444584295153618,
        -0.019985169172286987,
        -0.03968489170074463,
        0.004193773027509451,
        -0.06551185250282288,
        0.005189067218452692,
        -0.0032417664770036936,
        0.01748158410191536,
        0.01634136773645878,
        0.0025453148409724236,
        0.026446620002388954,
        -0.00528318015858531,
        -0.04398209601640701,
        0.004389017820358276,
        0.029156945645809174,
        -0.029075095430016518,
        -0.014196628704667091,
        -0.010793527588248253,
        0.0037630603183060884,
        -0.006193441804498434,
        -0.012116486206650734,
        0.03763418272137642,
        -0.03195863962173462,
        0.03644412010908127,
        0.015416806563735008,
        -0.035042840987443924,
        0.004068080335855484,
        -0.03145904093980789,
        0.029642581939697266,
        -0.04379571974277496,
        -0.02598177082836628,
        -0.05614802986383438,
        0.04552372545003891,
        0.03503458574414253,
        0.047580935060977936,
        0.03023313172161579,
        0.007014804519712925,
        -0.014679844491183758,
        0.0023398129269480705,
        0.032817546278238297,
        0.001456637866795063,
        -0.09598670899868011,
        -0.03523412346839905,
        0.058260854333639145,
        -0.02089003659784794,
        -0.012693151831626892,
        0.013551123440265656,
        -0.02357356809079647,
        0.010327352210879326,
        0.008712236769497395,
        0.014864714816212654,
        -0.05297556146979332,
        0.04564664885401726,
        -0.0419088713824749,
        -0.02170810103416443,
        0.03652159497141838,
        -0.0004907330730929971,
        0.037996988743543625,
        -0.011682519689202309,
        -0.014512947760522366,
        0.016863461583852768,
        -0.01886449009180069,
        0.00041054506436921656,
        -0.031114082783460617,
        -0.008560906164348125,
        -0.05541972815990448,
        -0.06915032118558884,
        0.00048636464634910226,
        -0.01588352583348751,
        0.015378400683403015,
        -0.04205279424786568,
        0.01761300303041935,
        0.03259969502687454,
        0.026089411228895187,
        -0.005170463118702173,
        -0.029360374435782433,
        -0.003456592094153166,
        -0.010034237988293171,
        -0.008756469935178757,
        0.016068436205387115,
        0.017009947448968887,
        -0.011077187024056911,
        0.0196326095610857,
        0.029606560245156288,
        0.057201988995075226,
        0.019467391073703766,
        0.045469511300325394,
        0.05130443349480629,
        0.01695866510272026,
        0.02687619812786579,
        0.055627550929784775,
        0.022768497467041016,
        -0.02940327674150467,
        0.004204964730888605,
        0.05033090338110924,
        -0.01545730046927929,
        0.006705779116600752,
        0.01895938441157341,
        0.02002001367509365,
        0.04580041021108627,
        -0.029465854167938232,
        -0.020475521683692932,
        -0.020832983776926994,
        -0.016766823828220367,
        -0.08100363612174988,
        -0.004184445831924677,
        -0.01912926882505417,
        0.0012200827477499843,
        0.04791146516799927,
        -0.021212514489889145,
        -0.022003188729286194,
        0.03242906555533409,
        -0.010839282535016537,
        -0.006080326158553362,
        -0.0010902824578806758,
        0.017618756741285324,
        0.009092114865779877,
        -0.015486473217606544,
        -0.015819797292351723,
        -0.0379180982708931,
        -0.0027636706363409758,
        -0.029309526085853577,
        -0.04951871931552887,
        -0.005742970854043961,
        -0.019167136400938034,
        -0.044334907084703445,
        0.02298717014491558,
        -0.02119157277047634,
        -0.028601977974176407,
        0.024080226197838783,
        -0.03263629600405693,
        -0.02560291811823845,
        -0.07433179020881653,
        -0.008937416598200798,
        -0.015358136966824532,
        0.030914485454559326,
        0.0022867934312671423,
        -0.0007416998851113021,
        -0.013609550893306732,
        -0.004528982564806938,
        -0.018863148987293243,
        0.029320329427719116,
        0.08042354136705399,
        -0.023553429171442986,
        0.019077904522418976,
        0.032800670713186264
      ]
# Cosine similarity with prefix: Cosine similarity: 0.9999999999986178
# Cosine similarity without prefix: Cosine similarity: 0.9999999999975963
print("Cosine similarity:", cosine_similarity(embedding_vector, es_vector))

Cosine similarity: 0.9999999999989528


In [ ]:
print("ES Vector (first 5):", es_vector[:5])
print("PyTorch Vector (first 5):", embedding_vector[:5])

ES Vector (first 5): [-0.012689272873103619, -0.005864003207534552, -0.011758002452552319, 0.05332026258111, 0.015246693976223469]
PyTorch Vector (first 5): [-0.012689327821135521, -0.005863994359970093, -0.011758034117519855, 0.05332023650407791, 0.015246723778545856]


To generate the embedding for the query inside Elasticsearch run this request in Dev Tools.

In [ ]:
POST _ml/trained_models/bge-m3/_infer
{
  "docs": [
    {
      "text_field": "Radio, TV, and Film Broadcasting History"
    }
  ]
}

SyntaxError: invalid syntax (<ipython-input-3-e41b1094ff34>, line 1)